# Using EcoFOCIpy to process raw field data

## DY2206 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [3]:
import yaml
import glob

import ecofocipy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import ecofocipy.io.ncCFsave as ncCFsave
import ecofocipy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [4]:
sample_data_dir = '/Users/bell/ecoraid/2022/CTDcasts/dy2206/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [5]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2206' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2206.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2022/CTDcasts/dy2206/rawconverted/ctd013.btl
Processing /

In [6]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,turbwetntu0,scan
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.00000,7.000000,7.000000,7.000000,7.000000,7.000000
mean,9.115157,118.897429,397.099143,31.549186,32.022686,25.159900,25.537629,8.802114,114.771571,383.479714,6.69750,2.651457,2.651629,32.154143,0.934600,14339.714286
std,0.498943,6.604712,21.858227,1.762330,0.509460,1.410728,0.411679,0.642388,7.668269,28.401081,1.74936,0.091814,0.092680,23.377049,0.534814,3353.548852
min,8.785100,114.901000,382.508000,27.553400,30.870100,21.961600,24.607200,8.349400,109.202000,363.536000,5.01670,2.573500,2.573100,3.217000,0.641300,9335.000000
25%,8.808700,115.258500,383.537500,32.181450,32.181150,25.659450,25.659300,8.389450,109.772500,365.282500,5.16955,2.590200,2.589850,15.060500,0.708400,12250.000000
50%,8.865500,116.045000,386.010000,32.231400,32.230900,25.708000,25.707600,8.429300,110.335000,367.017000,6.21350,2.605500,2.605000,30.489000,0.733500,14356.000000
75%,9.192650,118.700500,401.001000,32.232100,32.231750,25.709750,25.709500,9.111700,118.892500,396.750000,8.01300,2.706000,2.707650,46.189500,0.806750,16638.000000
max,10.152800,133.418000,442.099000,32.232400,32.232000,25.711300,25.711000,9.833800,126.534000,429.740000,9.28720,2.788800,2.788300,68.873000,2.137100,18911.000000


## Create BTL report file

In [7]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [8]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2206',
 'CruiseID_Historic': 'DY2206',
 'CruiseID_Alternates': 'DY2206',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2022, 5, 10),
 'EndDate': datetime.date(2022, 5, 19),
 'Project': 'FOCI',
 'ChiefScientist': 'Shaun Bell',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Mooring Survey',
 'CruiseYear': 2022,
 'ctdlogs_pdf_name': 'DY2206_CTDCastLogs.pdf'}

In [9]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 53827,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2206',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2206',
 'Project': 'FOCI Spring Mooring Survey',
 'StationNo_altname': 's1h1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 56,
 'LatitudeMin': 51.79,
 'LongitudeDeg': 164,
 'LongitudeMin': 1.91,
 'GMTDay': 11,
 'GMTMonth': 'May',
 'GMTYear': 2022,
 'GMTTime': 50340,
 'DryBulb': -99.9,
 'RelativeHumidity': 95,
 'WetBulb': -99.9,
 'Pressure': 1027,
 'SeaState': '',
 'Visibility': '',
 'WindDir': 275,
 'WindSpd': 14.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 72,
 'StationNameID': '70MC',
 'MaxDepth': 68,
 'InstrumentSerialNos': '',
 'Notes': 'Nuts paper label says DY2203, cap labeled Niskin/cast .. const O2 offset between sensors of about 5%\r\nfluor voltage is good, plot data is empty (calibration issue)\r\ndue to rosette setup it is possible first cast had color sampled backwards',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7',
 '

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [10]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [11]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,Pressure [dbar],turbidity,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
9.0,9.0419,118.438,393.694,32.2266,32.2261,25.7017,25.7014,8.5944,112.577,374.21,5.0167,2.6352,2.635,20.122,0.6413,15851.0,2022-05-11 06:06:00,ctd001


In [12]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'Pressure [dbar]', 'turbidity',
       'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [13]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [14]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [15]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 7)
Coordinates:
  * bottle              (bottle) float64 1.0 12.0 11.0 10.0 9.0 8.0 7.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 8.785 8.794 8.823 ... 9.042 9.343 10.15
    oxy_percentsat_ch1  (bottle) float64 114.9 115.1 115.5 ... 118.4 119.0 133.4
    oxy_conc_ch1        (bottle) float64 382.5 382.9 384.2 ... 393.7 408.3 442.1
    salinity_ch1        (bottle) float64 32.23 32.23 32.23 ... 32.23 27.55 32.14
    salinity_ch2        (bottle) float64 32.23 32.23 32.23 ... 32.23 30.87 32.14
    sigma_t_ch1         (bottle) float64 25.71 25.71 25.71 ... 25.7 21.96 25.62
    ...                  ...
    temperature_ch2     (bottle) float64 2.573 2.585 2.595 ... 2.635 2.78 2.788
    Pressure [dbar]     (bottle) float64 68.87 51.71 40.67 ... 20.12 9.999 3.217
    turbidity           (bottle) float64 2.137 0.871 0.7425 ... 0.7335 0.7226
    scan                (bottle) float64 9.335e+03 1.154e+04 ... 1.891e+04
    datetime            (bottle) datetime64[ns] 2022-05-11T06:01:29 ... 2022-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [16]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [17]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [18]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 7, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 12.0 11.0 10.0 9.0 8.0 7.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 8.785 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 114.9 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 382.5 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.23 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.23 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.71 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 2.573 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 68.87 ......
    turbidity           (latitude, longitude, time, bottle) float64 2.137 ......
    scan                (latitude, longitude, time, bottle) float64 9.335e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [19]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 7, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 12.0 11.0 10.0 9.0 8.0 7.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 8.785 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 114.9 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 382.5 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.23 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.23 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.71 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 2.573 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 68.87 ......
    turbidity           (latitude, longitude, time, bottle) float64 2.137 ......
    scan                (latitude, longitude, time, bottle) float64 9.335e+03...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      DY2206
    CruiseID_Historic:             DY2206
    CruiseID_Alternates:           DY2206
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2022-09-30T10:29:05Z
    date_modified:                 
    QC_indicator:                  unqcd

## Rare Bottle File Edits

<div class="warning" style='background-color:#ffcccb; color: #FF0000; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>WARNING</b></p>
<p style='margin-left:1em;'>bottle/niskin and rosette position should be the same but can be different (example, bottles are labeled sequentially but a rosette position is skipped due to balancing or other instruments.  On this cruise the following rosette positin was fired, while the bottles where labled differently</p>
cruise_data[cast]
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"><span style="font-weight:bold">Rosette</span></th>
    <th class="tg-0lax">1</th>
    <th class="tg-0lax">12</th>
    <th class="tg-0lax">11</th>
    <th class="tg-0lax">10</th>
    <th class="tg-0lax">9</th>
    <th class="tg-0lax">8</th>
    <th class="tg-0lax">7<br></th>
    <th class="tg-0lax">6</th>
    <th class="tg-0lax">5</th>
    <th class="tg-0lax">4</th>
    <th class="tg-0lax">3</th>
    <th class="tg-0lax">2</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax"><span style="font-weight:bold">Niskin</span><br></td>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">5</td>
    <td class="tg-0lax">6</td>
    <td class="tg-0lax">7</td>
    <td class="tg-0lax">8</td>
    <td class="tg-0lax">9</td>
    <td class="tg-0lax">10<br></td>
    <td class="tg-0lax">11</td>
    <td class="tg-0lax">12</td>
  </tr>
</tbody>
</table>
</div>

In [40]:
import pandas as pd
nisk_btl = pd.DataFrame(data=[[1,1],[2,12],[3,11],[4,10],[5,9],[6,8],[7,7],[8,6],[9,5],[10,4],[11,3],[12,2]],columns=['bottle','rosette']) 
for cast in cruise_data.keys():
    cruise_data[cast] = pd.merge(nisk_btl, cruise_data[cast], left_on='rosette', right_on='bottle').drop('rosette',axis=1).set_index('bottle')      

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [41]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history